# Code handout - Assignment 9 - Simulation of the car model

In this (optional) problem, you will implement the ordinary differential equation which defines the movement of the car you explored as a bond graph.
You will implement the state equations you extracted from the bond graph in task 2f.

First, we import some of the libraries that might be useful.

In [ ]:
import numpy as np
import scipy as sp
import sympy as sm
import matplotlib.pyplot as plt

The linear friction coefficient of the clutch is a function of the clutch signal $u_C$.
For simplicity, we will model the friction as

$$
R_C = R_{C, \max} u_C
$$

and let the signal $u_C$ follow a ramp.
This ramp will increase linearly from $0$ to $1$ on the time interval $[t_s, t_s + \Delta t]$
where $t_s$ is the start time of the ramping and $\Delta t$ is the ramp duration.

Below, we've  implemented such a function and vectorized it with numpy to allow it to work on whole arrays at a time.

In [ ]:
@np.vectorize
def ramp(current_time, ramp_start_time, ramping_duration):
    if current_time < ramp_start_time:
        return 0.0
    elif current_time <= ramp_start_time + ramping_duration:
        return (current_time - ramp_start_time) / ramping_duration
    else:
        return 1.0

We also try to plot the ramp for $t_s=0.2$ and $\Delta t = 0.6$ in order to verify that the implementation makes sense.
The ramp should start at $0.2$ and end at $0.8$.

In [ ]:
plt.plot(np.linspace(0, 1, num=101), ramp(np.linspace(0, 1, num=101), 0.2, 0.6))
plt.title("Ramp starting at 0.2 and ending at 0.8")

## Setting up the parameters for the car

We can define the parameters of the car with a Python dataclass.
That way, we can pass around an instantiation as a "configuration object" of sorts,
both to reduce the number of parameters each function must take, but also to ensure that all the vital parameters are set.
If we passed around a dictionary, we'd ideally have to check if all the required parameters were present, but since the object _is_ an instantiation, it _must have_ all the parameters defined!

Python dataclasses are more or less structs in C and implements some nice functions like `__init__` and `__repr__` which will let us write less boilerplate code and have a bigger focus on implementing the simulation.

In [ ]:
from dataclasses import dataclass

@dataclass
class CarParameters():
    # Moments of inertia -- engine/clutch
    J_E: float
    J_G1: float
    J_S: float
    # Engine/clutch parameters
    R_E: float
    R_C: float
    R_G1: float
    R_G2: float
    # Shaft deflection
    R_shaft_deflection: float
    C_shaft_deflection: float
    # Axle parameters
    N: float
    J_R: float
    J_L: float
    R_R: float
    R_L: float
    # Tire/road friction
    R_wr: float
    R_wl: float
    # Car parameters
    mass: float
    R_car: float
    rw: float

After defining the dataclass, we instantiate our car with the parameters from the assignment.
Needless to say, there's a lot of them.

In [ ]:
car_params = CarParameters(
    J_E = 10, J_G1 = 1, J_S = 2,
    R_E = 0.2, R_C = 70, R_G1 = 0.12, R_G2 = 0.15,
    R_shaft_deflection = 200, C_shaft_deflection = 1 / 5_000,
    N = 0.18, J_R = 2, J_L = 2, R_R = 0.2, R_L = 0.2,
    R_wr = 800, R_wl = 480,
    mass = 2500, R_car = 6, rw = 0.37
)

## TODO: Implementing the state equations

Here, we can write the code for the function $f$ in $\dot{y} = f(t, y)$, so that we can simulate the behavior of the system.
Our function will also take some extra inputs, like the car parameters and the torque- and clutch signals.

If your bond graph is correct, there should be 6 states relevant to the system, and thus 6 state equations you have to implement.

Good luck.

In [ ]:
def ode(car: CarParameters, u_T, u_c, y):

    omega_E = 0  #TODO: engine speed as function of states
    torque_lim = 0 #TODO: expression for the engine torque limit as reported by General Motors
    T_E = 0 #TODO: engine torque as given by torque signal u_T, described in equation in task 1a.

    dydt = np.zeros_like(y)

    return dydt

## Simulation

Now we can simulate the system, which we will do with a simple implementation of forward Euler.

In [ ]:
## Simulation parameters
t0, tf = 0, 30
h = 5e-4
y_ic = np.zeros((6,))
num_steps = int((tf - t0) / h) + 1

## Clutch behavior parameters
t_start_clutch = 10
dt_clutch = 2

ys = np.zeros((6, num_steps))
ts = np.zeros((1, num_steps))
ys[:, 0] = y_ic
ts[0, 0] = 0

## Controller parameters
K_p = 5
K_i = 0.6
engine_speed_ref = 2000
control_error = engine_speed_ref * (np.pi / 30) - ys[0, 0] / car_params.J_E
integral_error = 0

## Saving the control signals so we can look at them after 
# the simulation to see if they make sense.
uts = []
ucs = []

for i in range(1, num_steps):
    # Engine speed controller
    u_T = control_error * K_p + integral_error * K_i
    # Clutch control signal
    ts[:, i] = ts[:, i-1] + h
    u_c = ramp(ts[0, i], t_start_clutch, dt_clutch)
    uts.append(u_T)
    ucs.append(u_c)
    # Forward Euler step
    ys[:, i] = ys[:, i-1] + h * ode(car_params, u_T, u_c, ys[:, i-1])

    control_error = engine_speed_ref * (np.pi / 30) - ys[0, i] / car_params.J_E
    integral_error = np.max([integral_error + control_error * h, 1.0])

## Plotting the results

The graphs shown here will of course depend on the order you put the states in the state vector.
If you put them in the same sequence as us, the plots will make sense, if not, I'm sure you'll figure it out.

Good luck.

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12,12))

ax1.set_title("Engine speed (rpm)")
ax1.plot(ts[0, :], ys[0, :] * 60 / (2*np.pi) / car_params.J_E, label="Engine")
ax1.plot(ts[0, :], ys[1, :] * 60 / (2*np.pi) / car_params.J_G1, label="Gear (engine-side)")

ax2.set_title("Forward speed of the car (km/h)")
ax2.plot(ts[0, :], ys[5, :] * 3.6 / car_params.mass)

ax3.set_title("Wheel speed (km/h)")
ax3.plot(ts[0, :], ys[3, :] * 3.6 * car_params.rw/car_params.J_R, label="Right")
ax3.plot(ts[0, :], ys[4, :] * 3.6 * car_params.rw/car_params.J_L, label="Left")

plt.show()

plt.title("Deflection of drive shaft (rads)")
plt.plot(ts[0, :], ys[2, :])
plt.show()